In [1]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn.preprocessing import label_binarize
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
from catboost import *
from sklearn.metrics import roc_curve, auc
from itertools import cycle
from scipy import interp
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_TOF_MCL.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Ionization Potential',#0
                      'Electronegativity',#1
                      'Number of d electrons',#2
                      'Graphene/Carbon Nanosheets or other 2D Structures',#3
                      'Carbon Nanofiber/Nanotubes',#4
                      'Biomass or other Organic Derived',#5  
                      'Main Transition Metal Content (wt. %)',#6
                      'Nitrogen Cotent (wt. %)',#7
                      'Metal-N Coordination Number (XAS)',#8    
                      'Pyridinic N Ratio',#9
                      'Pyrrolic N Ratio',#10
                      'Raman ID/IG Ratio',#11
                      'BET Surface Area (m2/g)',#12
                      'Pyrolysis Temperature (°C)',#13
                      'Pyrolysis Time (h)',#14
                      'Rising Rate (°C min-1)',#15
                      'Flow Cell/H-type Cell',#16
                      'Electrolyte Concentration (M)',#17
                      'Catalyst Loading (mg cm-2)',#18
                      'Carbon Paper/Glassy Carbon',#19
                      'Electrolyte pH',#20
                      'TOF_MCL_2_4'#21
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:21]
raw_power=raw_data.iloc[:,21]
print('ready')
nb_classes=3

Using matplotlib backend: Qt5Agg
ready


In [11]:
from catboost import *
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    Y_test_labeled = label_binarize(y_test, classes=[i for i in range(nb_classes)])
    y_score=best_model.predict_proba(X_test)
#     y_score=y_score[:,1]
#     print(y_score)
    auc_curve(Y_test_labeled,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
def auc_curve(y_label, y_pre,algorithm_name):
#     y_label = y_label + 1
#     y_pre = y_pre + 1
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(nb_classes):
        fpr[i], tpr[i], _ = roc_curve(y_label[:, i], y_pre[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_label.ravel(), y_pre.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(nb_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(nb_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= nb_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])



    # Plot all ROC curves
    lw = 2
    fig=plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(nb_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Multi-Class Classification) of %s' %algorithm_name)
    plt.legend(loc="lower right")
    plt.savefig('ROC Curve of %s TOFMCL 3D.png' %algorithm_name)
def shap_plot_multi(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1]), SHAP_INPUT,max_display=100)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
    elif algorithm_name=='LightGBM':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)        
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
def shap_plot_0(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[0]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
def shap_plot_1(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[1]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[1]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[1]).mean(0)
        print(global_importances)
def shap_plot_2(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[2]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[2]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[2]).mean(0)
        print(global_importances)

In [7]:
seed=1
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.1,random_state=seed)
#########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0,bootstrap_type='Bernoulli')
param_cat = {
 'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
 'n_estimators':[50,100,200,400],
 "boosting_type":["Plain"],
 'max_depth':[5,7,9,11],
 'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
  'boosting_type':['gbdt','rf'],
    'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
    'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
     'n_estimators':[50,100,200,400],
    'max_depth':[5,7,9,11,-1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
 'n_estimators':[50,100,200,400],
 'max_depth':[5,7,9,11,16],
 'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
  'n_estimators':[50,100,200,400],
 'max_depth':[3,5,7,9,11,13,16],
 'criterion':['friedman_mse','mae','mse'],
 'max_features':['auto','sqrt','log2'],
 'loss':['deviance', 'exponential']
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
      'n_estimators':[10,50,100,200,400],
     'max_depth':[3,5,7,9,11,None],
     'criterion':['gini','entropy'],
     'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
        'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
        'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
      'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
    'n_estimators':[50,100,200,400]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')


In [5]:
train_TOFMCL=pd.concat([X_train,y_train],axis=1)
test_TOFMCL=pd.concat([X_test,y_test],axis=1)
train_TOFMCL.to_csv("train_TOFMCL.csv", index_label="index_label")
test_TOFMCL.to_csv("test_TOFMCL.csv", index_label="index_label")

In [13]:
##########LightGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt'],
 'learning_rate':[1],
  'n_estimators':[50],
 'subsample':[0.4],
 'max_depth':[5],
 'reg_alpha':[0.01],
 'reg_lambda':[0]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

start
Best Classifier: {} Best Score: 0.8153167602245389
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        88
           1       1.00      1.00      1.00       327
           2       1.00      1.00      1.00        18

    accuracy                           1.00       433
   macro avg       1.00      1.00      1.00       433
weighted avg       1.00      1.00      1.00       433



scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead


              precision    recall  f1-score   support

           0       0.89      0.50      0.64        16
           1       0.74      0.97      0.84        30
           2       1.00      0.33      0.50         3

    accuracy                           0.78        49
   macro avg       0.88      0.60      0.66        49
weighted avg       0.81      0.78      0.75        49

0.7755102040816326
LightGBM
[ 93 182  68  33  13   0 986 733 276 545 472 456 665 282 114 210  75  44
 376  22  46]


In [14]:
shap_plot_0(model_LightGBMClassifier,param_light,'LightGBM')

LightGBM


 99%|===================| 1438/1446 [00:14<00:00]        

[0.01132402 0.8166606  0.11091536 0.02276346 0.0302935  0.
 1.50077325 0.26742055 0.26316762 0.16189225 0.29792282 0.24878463
 0.32114117 0.323368   0.1144842  0.09159662 0.         0.10816252
 0.41459635 0.02099891 0.02348022]


In [16]:
shap_plot_1(model_LightGBMClassifier,param_light,'LightGBM')

LightGBM


 94%|=================== | 1363/1446 [00:13<00:00]       

[0.03593038 0.4176627  0.07315516 0.01971146 0.06474951 0.01796068
 0.48937833 0.31161941 0.17047535 0.34793359 0.18886903 0.17130963
 0.42551555 0.37084609 0.10101471 0.14610546 0.         0.01525504
 0.08723726 0.0086806  0.05833854]


In [17]:
shap_plot_2(model_LightGBMClassifier,param_light,'LightGBM')

LightGBM


 96%|=================== | 1382/1446 [00:14<00:00]       

[2.36575566e-01 1.51252329e-01 3.92362065e-02 1.15557310e-04
 0.00000000e+00 0.00000000e+00 1.18631250e+00 3.32225637e-01
 1.29885483e-01 2.18421414e-01 1.10485687e-01 7.87520548e-02
 2.91078418e-01 2.55226385e-02 4.85757375e-02 5.14364107e-02
 1.84828101e-01 2.94522906e-05 1.63155343e+00 1.30110052e-02
 4.39435440e-02]


In [12]:
shap_plot_multi(model_LightGBMClassifier,param_light,'LightGBM')

LightGBM


AttributeError: 'TreeEnsemble' object has no attribute 'values'

In [19]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.25],
 'n_estimators':[50],
 'max_depth':[11],
 'subsample':[0.7],
 'reg_alpha':[0.01],
 'reg_lambda':[0.01]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

start
Best Classifier: {'booster': 'gbtree', 'learning_rate': 0.25, 'max_depth': 11, 'n_estimators': 50, 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'subsample': 0.7} Best Score: 0.8035017375033414


scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       313
           2       1.00      1.00      1.00        20

    accuracy                           1.00       433
   macro avg       1.00      1.00      1.00       433
weighted avg       1.00      1.00      1.00       433

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      0.98      0.99        44
           2       0.50      1.00      0.67         1

    accuracy                           0.98        49
   macro avg       0.83      0.99      0.89        49
weighted avg       0.99      0.98      0.98        49

0.9795918367346939
XGBoost
[0.046489   0.06863233 0.05284328 0.05289664 0.02747066 0.07511106
 0.08312552 0.04905857 0.04357879 0.04199241 0.02976919 0.0314338
 0.03616277 0.06940895 0.03613929 0.03705021 0.06095274 0.02871224
 0.05386861

In [20]:
shap_plot_0(model_XGBClassifier,param_xg,'XGBoost')

XGBoost
[0.06535396 0.66046987 0.1333882  0.01715981 0.01395146 0.00571751
 1.23869727 0.201422   0.12092518 0.12610314 0.23700581 0.24635778
 0.32053303 0.2937177  0.04636378 0.07246652 0.         0.03071402
 0.32397963 0.00714986 0.05730226]


In [21]:
shap_plot_1(model_XGBClassifier,param_xg,'XGBoost')

XGBoost
[0.08509234 0.28631622 0.07826487 0.02517312 0.01350372 0.02557901
 0.46512055 0.27019587 0.09601048 0.20798453 0.15905295 0.14864613
 0.29333524 0.2624804  0.08413663 0.13422298 0.0107461  0.02582098
 0.15074727 0.0330624  0.05678398]


In [24]:
shap_plot_2(model_XGBClassifier,param_xg,'XGBoost')

XGBoost
[0.00894828 0.03050933 0.00515887 0.         0.         0.
 0.8322222  0.15518842 0.12305332 0.11565768 0.10220103 0.02894086
 0.16371496 0.06271433 0.01837712 0.06054756 0.05504554 0.
 0.74607821 0.00356229 0.09742016]


In [25]:
shap_plot_multi(model_XGBClassifier,param_xg,'XGBoost')

XGBoost


In [26]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0,bootstrap_type='Bernoulli')
param_cat = {
 'learning_rate':[0.25],
 'n_estimators':[50],
 "boosting_type":["Plain"],
 'max_depth':[7],
 'subsample':[0.9],
    'reg_lambda':[0.01]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

start
Best Classifier: {'boosting_type': 'Plain', 'learning_rate': 0.25, 'max_depth': 7, 'n_estimators': 50, 'reg_lambda': 0.01, 'subsample': 0.9} Best Score: 0.8082865543972201
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       313
           2       1.00      1.00      1.00        20

    accuracy                           1.00       433
   macro avg       1.00      1.00      1.00       433
weighted avg       1.00      1.00      1.00       433

              precision    recall  f1-score   support

           0       0.67      0.50      0.57         4
           1       0.96      0.98      0.97        44
           2       1.00      1.00      1.00         1

    accuracy                           0.94        49
   macro avg       0.87      0.83      0.85        49
weighted avg       0.93      0.94      0.93        49

0.9387755102040817
CatBoost
[ 4.03922466  4.21494319  5.7997

scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead


In [27]:
shap_plot_0(model_CatClassifier,param_cat,'CatBoost')

CatBoost
[0.13957615 0.19425297 0.205807   0.03239226 0.00960438 0.00792466
 0.5788552  0.14827774 0.1411862  0.12285437 0.1137175  0.16641786
 0.17023254 0.19303377 0.07687125 0.07096162 0.02478764 0.05700155
 0.33388054 0.0227284  0.03931203]


In [28]:
shap_plot_1(model_CatClassifier,param_cat,'CatBoost')

CatBoost
[0.13817912 0.19445972 0.20292496 0.01886157 0.01074085 0.00451635
 0.56822999 0.15764429 0.14217378 0.12265568 0.1160882  0.18307446
 0.17115033 0.17728127 0.08947215 0.08182225 0.05598559 0.06826475
 0.29142488 0.02433868 0.06857625]


In [29]:
shap_plot_2(model_CatClassifier,param_cat,'CatBoost')

CatBoost
[0.13664685 0.18697482 0.20009322 0.0262835  0.01190404 0.01235377
 0.54829496 0.17072186 0.11529737 0.1326679  0.13860238 0.17982819
 0.16501165 0.19363148 0.07930614 0.06848395 0.03826074 0.05840805
 0.31579643 0.02406742 0.05317455]


In [30]:
shap_plot_multi(model_CatClassifier,param_cat,'CatBoost')

CatBoost


In [ ]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[1],
  'n_estimators':[100],
 'max_depth':[9],
 'criterion':['mse'],
 'max_features':['auto'],
 'loss':['deviance']
}

In [ ]:
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

In [31]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
      'n_estimators':[50],
     'criterion':['entropy'],
     'max_depth': [9],
     'max_features':['auto']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')

start
Best Classifier: {'criterion': 'entropy', 'max_depth': 9, 'max_features': 'auto', 'n_estimators': 50} Best Score: 0.7898957497995189
              precision    recall  f1-score   support

           0       1.00      0.89      0.94       100
           1       0.96      1.00      0.98       313
           2       1.00      0.95      0.97        20

    accuracy                           0.97       433
   macro avg       0.99      0.95      0.97       433
weighted avg       0.97      0.97      0.97       433


scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead



              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.96      1.00      0.98        44
           2       1.00      1.00      1.00         1

    accuracy                           0.96        49
   macro avg       0.99      0.83      0.88        49
weighted avg       0.96      0.96      0.95        49

0.9591836734693877
Random Forest
[0.03593535 0.03448052 0.03148879 0.00583281 0.00574446 0.0048983
 0.21285444 0.10119102 0.05100105 0.06122213 0.05570763 0.0532571
 0.07620486 0.05677921 0.02693375 0.03539242 0.01765068 0.01070004
 0.08153075 0.00731733 0.03387735]


In [32]:
shap_plot_0(model_RandomForestClassifier,param_RF,'Random Forest')

Random Forest
[0.01470068 0.02871847 0.0197333  0.00146787 0.00204928 0.00112876
 0.08575155 0.0274723  0.01366239 0.01541535 0.01363201 0.01037626
 0.0213064  0.02773078 0.00564534 0.00704237 0.00189864 0.00345756
 0.02501479 0.00302088 0.00818331]


In [33]:
shap_plot_1(model_RandomForestClassifier,param_RF,'Random Forest')

Random Forest
[0.01367715 0.02548913 0.01764782 0.00164329 0.0019794  0.00113492
 0.06273928 0.03257124 0.01275189 0.0165626  0.0141511  0.01125806
 0.02301667 0.0289365  0.00680624 0.00817824 0.00152027 0.00356975
 0.02218361 0.00348606 0.00780392]


In [34]:
shap_plot_2(model_RandomForestClassifier,param_RF,'Random Forest')

Random Forest
[6.29737788e-03 5.61440728e-03 3.08753202e-03 4.06005782e-04
 7.84002701e-04 8.08575387e-05 3.99955643e-02 6.89919961e-03
 7.91397605e-03 3.70672466e-03 3.62263945e-03 5.04781651e-03
 5.56997807e-03 4.60728614e-03 2.67042374e-03 3.35454314e-03
 2.67226608e-03 1.02881729e-03 2.66077579e-02 9.29716672e-04
 2.95442451e-03]


In [35]:
shap_plot_multi(model_RandomForestClassifier,param_RF,'Random Forest')

Random Forest


In [36]:
###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
        'max_depth':[5],
        'criterion' : ['entropy'],
        'splitter' : [ "random"],
        'max_features':['auto']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')

start
Best Classifier: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.7689120556001069
              precision    recall  f1-score   support

           0       0.73      0.30      0.43       100
           1       0.77      0.96      0.86       313
           2       0.00      0.00      0.00        20

    accuracy                           0.77       433
   macro avg       0.50      0.42      0.43       433
weighted avg       0.73      0.77      0.72       433

              precision    recall  f1-score   support

           0       0.20      0.25      0.22         4
           1       0.91      0.91      0.91        44
           2       0.00      0.00      0.00         1

    accuracy                           0.84        49
   macro avg       0.37      0.39      0.38        49
weighted avg       0.83      0.84      0.83        49

0.8367346938775511
Extra Tree
[0.03616244 0.         0.09978657 0.07045893 0.         0.
 0.36518

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [37]:
shap_plot_0(model_ExtraTreeClassifier,param_ET,'Extra Tree')

Extra Tree
[0.00694963 0.         0.02011061 0.00176003 0.         0.
 0.07296932 0.0284549  0.02649758 0.         0.         0.
 0.         0.         0.         0.         0.         0.00552444
 0.         0.         0.00193465]


In [38]:
shap_plot_1(model_ExtraTreeClassifier,param_ET,'Extra Tree')

Extra Tree
[0.00694963 0.         0.01480622 0.00176003 0.         0.
 0.0524834  0.02671473 0.02401669 0.         0.         0.
 0.         0.         0.         0.         0.         0.00552444
 0.         0.         0.00193465]


In [39]:
shap_plot_2(model_ExtraTreeClassifier,param_ET,'Extra Tree')

Extra Tree
[0.         0.         0.00530438 0.         0.         0.
 0.02057958 0.00174016 0.00249817 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [40]:
shap_plot_multi(model_ExtraTreeClassifier,param_ET,'Extra Tree')

Extra Tree


In [41]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
        'max_depth':[5],
        'criterion' : ['entropy'],
        'splitter' : [ "random"],
        'max_features':['auto']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')

start
Best Classifier: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.7689120556001069


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead


              precision    recall  f1-score   support

           0       0.73      0.30      0.43       100
           1       0.77      0.96      0.86       313
           2       0.00      0.00      0.00        20

    accuracy                           0.77       433
   macro avg       0.50      0.42      0.43       433
weighted avg       0.73      0.77      0.72       433

              precision    recall  f1-score   support

           0       0.20      0.25      0.22         4
           1       0.91      0.91      0.91        44
           2       0.00      0.00      0.00         1

    accuracy                           0.84        49
   macro avg       0.37      0.39      0.38        49
weighted avg       0.83      0.84      0.83        49

0.8367346938775511
Decision Tree
[0.03616244 0.         0.09978657 0.07045893 0.         0.
 0.36518019 0.14235783 0.19200244 0.         0.         0.
 0.         0.         0.         0.         0.         0.02359267
 0.         0.      

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [42]:
shap_plot_0(model_DecisionTreeClassifier,param_DT,'Decision Tree')

Decision Tree
[0.00694963 0.         0.02011061 0.00176003 0.         0.
 0.07296932 0.0284549  0.02649758 0.         0.         0.
 0.         0.         0.         0.         0.         0.00552444
 0.         0.         0.00193465]


In [43]:
shap_plot_1(model_DecisionTreeClassifier,param_DT,'Decision Tree')

Decision Tree
[0.00694963 0.         0.01480622 0.00176003 0.         0.
 0.0524834  0.02671473 0.02401669 0.         0.         0.
 0.         0.         0.         0.         0.         0.00552444
 0.         0.         0.00193465]


In [44]:
shap_plot_2(model_DecisionTreeClassifier,param_DT,'Decision Tree')

Decision Tree
[0.         0.         0.00530438 0.         0.         0.
 0.02057958 0.00174016 0.00249817 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [45]:
shap_plot_multi(model_DecisionTreeClassifier,param_DT,'Decision Tree')

Decision Tree


In [46]:
###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
      'learning_rate':[0.001],
    'n_estimators':[50]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

start
Best Classifier: {'learning_rate': 0.001, 'n_estimators': 50} Best Score: 0.759796845763165


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


              precision    recall  f1-score   support

           0       0.76      0.31      0.44       100
           1       0.77      0.97      0.86       313
           2       0.00      0.00      0.00        20

    accuracy                           0.77       433
   macro avg       0.51      0.43      0.43       433
weighted avg       0.73      0.77      0.72       433

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.94      1.00      0.97        44
           2       0.00      0.00      0.00         1

    accuracy                           0.94        49
   macro avg       0.65      0.50      0.54        49
weighted avg       0.92      0.94      0.92        49

0.9387755102040817
AdaBoost
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [47]:
shap_plot_0(model_AdaBoostClassifier,param_Ada,'AdaBoost')

AdaBoost
[0.         0.         0.         0.         0.         0.
 0.07431451 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [48]:
shap_plot_1(model_AdaBoostClassifier,param_Ada,'AdaBoost')

AdaBoost
[0.         0.         0.         0.         0.         0.
 0.06728274 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [49]:
shap_plot_2(model_AdaBoostClassifier,param_Ada,'AdaBoost')

AdaBoost
[0.         0.         0.         0.         0.         0.
 0.00703177 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]


In [50]:
shap_plot_multi(model_AdaBoostClassifier,param_Ada,'AdaBoost')

AdaBoost
